<h1><center>NHL DATA SCRAPE AND VISUALIZATION</center></h1>
<h2><center>A Learning Project</center></h2>

The following is an attempt to learn pandas, numpy, and beautifulsoup in order to create a repository of data scrapped from the web, clean it, and then process it to draw and test assumptions against that data. This will utilize the NHL API along with the NHL Play by Play sheet. One of the reasons to use a Jupyter Notebook is to avoid multiple calls to the mentioned websites but still continue to work with the data they provide.

First we import the required python libraries. 
- BeautifulSoup will allow us to manipulate the html files from the NHL Play by Play 
- NHLget is a self made module which accesses the NHL API and the NHL Play by Play pages. It has three functions: One to return a list of games played during the provided time period (NHL API). One to return game information about a specific game (NHL API), and the third to provide play by play information (NHL Play by Play).

In [1]:
import NHLget

In [2]:
soup = NHLget.get_player_on_ice_info('2021020001')

In [3]:
pages = soup.select('.page')

all_plays = NHLget.clean_play_by_play(pages)

In [4]:
game_json = NHLget.get_game_info('2021020001')

In [5]:
plays = game_json['liveData']['plays']['allPlays']

play_id_dict = {'GAME_SCHEDULED': 'PGSTR', 'PERIOD_READY': 'PGEND', 
'PERIOD_START': 'PSTR', 'FACEOFF': 'FAC', 'HIT': 'HIT', 'STOP': 'STOP', 
'SHOT': 'SHOT', 'TAKEAWAY': 'TAKE', 'BLOCKED_SHOT': 'BLOCK', 
'MISSED_SHOT': 'MISS', 'GIVEAWAY': 'GIVE', 'PERIOD_END': 'PEND', 
'PERIOD_OFFICIAL': '', 
  'GOAL': 'GOAL', 
  'PENALTY': 'PENL', 
  'GAME_END': 'GEND', 
  'GAME_OFFICIAL': ''}


In [57]:
test_API_plays = plays[:5]
test_PxP_plays = all_plays[:6]

# playId, period, strength, [Time Elapsed, Time Remaining], Event, Description, Away Team Players on Ice, Home Team Players on Ice, players involved, dateTime, coordinates

track_i = 0
for mainPlay in all_plays:
  if mainPlay[0] == '#':
    continue
  if mainPlay[3][0][0:2] == '0:':
    mainPlay[3][0] = '00:' + mainPlay[3][0][-2:]
  for ind in range(track_i, len(plays)):
    if all([
        play_id_dict[plays[ind]['result']['eventTypeId']] == mainPlay[4],
        plays[ind]['about']['period'] == int(mainPlay[1]),
        plays[ind]['about']['periodTime'] == mainPlay[3][0]
    ]):
      if 'players' in plays[ind]:
        mainPlay.append(plays[ind]['players'][0])
        if len(plays[ind]['players']) > 1:
          mainPlay.append(plays[ind]['players'][1])
          if len(plays[ind]['players']) > 2:
            mainPlay.append(plays[ind]['players'][2])
            if len(plays[ind]['players']) > 3:
              mainPlay.append(plays[ind]['players'][3])
      mainPlay.append(plays[ind]['about']['dateTime'])
      mainPlay.append(plays[ind]['coordinates'])
      track_i = ind + 1

for play in all_plays:
  print(play)


['#', 'Per', 'Str', [], 'Event', 'Description', 'PIT On Ice', 'TBL On Ice']
['1', '1', '', ['00:00', '20:00'], 'PGSTR', '', {}, {}, '2021-10-12T22:45:48Z', '2021-10-12T22:45:48Z', '2021-10-12T22:45:48Z', {}, '2021-10-12T22:45:48Z', {}]
['2', '1', '', ['00:00', '20:00'], 'PGEND', '', {}, {}, '2021-10-12T23:41:26Z', '2021-10-12T23:41:26Z', '2021-10-12T23:41:26Z', {}, '2021-10-12T23:41:26Z', {}]
['3', '1', '', ['00:00', '20:00'], 'ANTHEM', '', {}, {}]
['4', '1', '', ['00:00', '20:00'], 'PSTR', 'Period Start- Local time: 7:43 EDT', {'C': '77', 'R': '17', 'L': '43', 'D1': '8', 'D2': '58', 'G': '35'}, {'C': '21', 'R': '86', 'L': '18', 'D1': '27', 'D2': '81', 'G': '88'}, '2021-10-12T23:44:06Z', '2021-10-12T23:44:06Z', '2021-10-12T23:44:06Z', {}, '2021-10-12T23:44:06Z', {}]
['5', '1', 'EV', ['00:00', '20:00'], 'FAC', 'PIT won Neu. Zone - PIT #77 CARTER vs TBL #21 POINT', {'C': '77', 'R': '17', 'L': '43', 'D1': '8', 'D2': '58', 'G': '35'}, {'C': '21', 'R': '86', 'L': '18', 'D1': '27', 'D2': '81